In [2]:
#data science stuff
import numpy as np
import pandas as pd
import pprint
import configparser
config = configparser.ConfigParser()
config.read('config.INI')
import time
import math

%run osuApiRequests.ipynb
%run readBans.ipynb

#requests
import requests
print("dab")

google sheets authentication success
dab


In [4]:
#return a df of overall statistics
def createOverview(bans, team_size, leaderboards_in_stage, maps_info, mods_for_each_map, num_matches, leaderboard_spreadsheetId):
    
    #---loop through each map and add the necessary info---
    overview = pd.DataFrame(columns=['Map', 'Picks', 'Bans', 'Total %', 'Average\nScore', 'Average\nAccuracy'])
    row_num = 2 #the row where appending starts
    for map_id in maps_info:
        
        #Get artist and name
        map_artist_and_diff = "%s - %s [%s]" % (maps_info[map_id]['artist'], maps_info[map_id]['name'], maps_info[map_id]['diff name'])
        
        #Get pick count
        #pick_count = "=CEILING(COUNT(IMPORTRANGE(\"https://docs.google.com/spreadsheets/d/%s\", \"%s!E8:E1000\")) / %d)" \
        #                % (leaderboard_spreadsheetId, maps_info[map_id]['name'], team_size * 2) 
        pick_count = "=COUNTUNIQUE(IMPORTRANGE(\"https://docs.google.com/spreadsheets/d/%s\", \"%s!J8:J1000\"))" \
                        % (leaderboard_spreadsheetId, maps_info[map_id]['name'])
        
        #Get ban count on each map
        ban_count = 0
        map_row = []
        map_row = bans.loc[
            bans['Map'].str.contains(map_id, case=False)
        ]
        if len(map_row.index) == 1:
            map_row = map_row.reset_index(drop=True)
            ban_count = int(map_row['Ban Count'][0])
        
        #Calculate total participation rate
        participation_rate = "=(INDIRECT(\"R[0]C[-2]\", FALSE) + INDIRECT(\"R[0]C[-1]\", FALSE)) / %d" \
                                % (num_matches)
        maps_info[map_id]['participation rate'] = participation_rate
        
        #Get avg score, avg acc
        avg_score = "=ROUND(AVERAGE(IMPORTRANGE(\"https://docs.google.com/spreadsheets/d/%s\", \"%s!E8:E1000\")), 0)" \
                        % (leaderboard_spreadsheetId, maps_info[map_id]['name']) 
        avg_acc = "=ROUND(AVERAGE(IMPORTRANGE(\"https://docs.google.com/spreadsheets/d/%s\", \"%s!G8:G1000\")), 4)" \
                        % (leaderboard_spreadsheetId, maps_info[map_id]['name']) 
        #avg_score = round(leaderboards_in_stage[map_id]['scores']['Score'].mean(), 0)
        #avg_acc = round(leaderboards_in_stage[map_id]['scores']['Accuracy'].mean(), 2)
        maps_info[map_id]['avg_score'] = avg_score
        maps_info[map_id]['avg_acc'] = avg_acc
        
        
        #Append new row to overview df
        overview = overview.append({
            'Map': map_artist_and_diff,
            'Picks': pick_count,
            'Bans': ban_count,
            'Total %': participation_rate,
            'Average\nScore': avg_score,
            'Average\nAccuracy': avg_acc
        }, ignore_index=True)
        
        row_num += 1
        
    return overview

In [ ]:
#(deprecated) get ban count on each map
"""
map_row = []
map_row = bans.loc[
    bans['Map'].str.contains(map_id, case=False)
]
if len(map_row.index) == 1:
    map_row = map_row.reset_index(drop=True)
    ban_count = int(map_row['Ban Count'][0])
maps_info[map_id]['ban count'] = ban_count
"""

In [1]:
#return df of best scores
def createBestPlays(maps_info, leaderboard_spreadsheetId):
    bestPlays = pd.DataFrame(columns=['Map', 'Best Score'])
    for map_id in maps_info:
        map_artist_and_diff = "%s - %s [%s]" % (maps_info[map_id]['artist'], maps_info[map_id]['name'], maps_info[map_id]['diff name'])
        best_score = "=IMPORTRANGE(\"https://docs.google.com/spreadsheets/d/%s\", \"%s!B8:J8\")" \
                    % (leaderboard_spreadsheetId, maps_info[map_id]['name'])
        bestPlays = bestPlays.append({
            'Map': map_artist_and_diff,
            'Best Score': best_score
        }, ignore_index=True)
    return bestPlays